In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, InputLayer, BatchNormalization, Input
from tensorflow.keras.losses import MeanSquaredError, Huber, MeanAbsoluteError, BinaryCrossentropy
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

### Preparing Data

In [ ]:
dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, shuffle_files=True, split=['train'])

In [ ]:
dataset

In [ ]:
dataset_info

In [ ]:
for data in dataset[0].take(1):
  print(data)
  break

In [ ]:
def splits(dataset, train_ratio, val_ratio, test_ratio):

  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(train_ratio*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(train_ratio*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(val_ratio*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(val_ratio*DATASET_SIZE))

  return train_dataset, val_dataset, test_dataset

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)
print(list(train_dataset.take(1).as_numpy_iterator()),
      list(val_dataset.take(1).as_numpy_iterator()), list(test_dataset.take(1).as_numpy_iterator()))

### Data Visualization

In [ ]:
for i, (image, label) in enumerate(train_dataset.take(16)):
  ax = plt.subplot(4, 4, i + 1)
  plt.imshow(image)
  plt.title(dataset_info.features['label'].int2str(label))
  plt.axis('off')

### Data Preprocessing

In [ ]:
IM_SIZE = 224
def resizing_rescaling(image, label):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

In [ ]:
train_dataset = train_dataset.map(resizing_rescaling)
val_dataset = val_dataset.map(resizing_rescaling)
test_dataset = test_dataset.map(resizing_rescaling)
train_dataset

In [ ]:
for image, label in train_dataset.take(1):
  print(image, label)

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

### Model Subclassing

In [ ]:
class FeatureExtractor(Layer):
  def __init__(self):
    super(FeatureExtractor, self).__init__()

    self.conv_1 = Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu')
    self.batch_1 = BatchNormalization()
    self.pool_1 = MaxPool2D(pool_size=2, strides=2)
    self.conv_2 = Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu')
    self.batch_2 = BatchNormalization()
    self.pool_2 = MaxPool2D(pool_size=2, strides=2)

  def call(self, x, training):
    x = self.conv_1(x)
    x = self.batch_1(x)
    x = self.pool_1(x)
    x = self.conv_2(x)
    x = self.batch_2(x)
    x = self.pool_2(x)

    return x


feature_sub_classed = FeatureExtractor()

In [ ]:
class LenetModel(Model):
  def __init__(self):
    super(LenetModel, self).__init__()

    self.feature_extractor = FeatureExtractor()
    self.flatten = Flatten()
    self.dense_1 = Dense(100, activation='relu')
    self.batch_1 = BatchNormalization()
    self.dense_2 = Dense(10, activation='relu')
    self.batch_2 = BatchNormalization()
    self.dense_3 = Dense(1, activation='sigmoid')

  def call(self, x, training):
    x = self.feature_extractor(x)
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.batch_1(x)
    x = self.dense_2(x)
    x = self.batch_2(x)
    x = self.dense_3(x)

    return x


lenet_model_sub = LenetModel()
lenet_model_sub(tf.zeros([1,224,224,3]))
lenet_model_sub.summary()

In [ ]:
lenet_model_sub.compile(optimizer=Adam(learning_rate=0.01),
              loss=BinaryCrossentropy(),
              metrics='accuracy',
              )

In [ ]:
history = lenet_model_sub.fit(train_dataset, validation_data=val_dataset, epochs=20, verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'val_loss'])
plt.show()

### Model Evaluation and Testing

In [ ]:
test_dataset = test_dataset.batch(1)

In [ ]:
lenet_model_sub.evaluate(test_dataset)

In [ ]:
lenet_model_sub.predict(test_dataset.take(1))[0][0]

In [ ]:
def parasite_or_not(x):
  if (x < 0.5):
    return str('P')
  else:
    return str('U')

In [ ]:
parasite_or_not(lenet_model_sub.predict(test_dataset.take(1))[0][0])

In [ ]:
for i, (image, label) in enumerate(test_dataset.take(9)):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image[0])
  plt.title(str(parasite_or_not(label.numpy()[0])) + ':' + str(parasite_or_not(lenet_model_sub.predict(image)[0][0])))

  plt.axis('off')

In [ ]:
lenet_model_sub.save('lenet_model_sub.keras')

In [ ]:
lenet_loaded_model = tf.keras.models.load_model('lenet_model_sub.keras')
lenet_loaded_model.summary()